# Generar reporte en excel

Generar un reporte en excel con imagenes incrustadas. Necesita como entrada una tabla con la lista de archivos para generar reporte. 

El ejemplo funciona con una lista generada por el software Grand Perspective.

A partir de la lista se genera un reporte en excel, conla imagen en la primera columna

In [1]:
import os
import datetime
import numpy as np
import pandas as pd
from pathlib import Path
from PIL import Image

In [ ]:
ruta_tabla = 'entrada/registros.csv'
stamp = datetime.datetime.now()

outfile = f"salida/reporte_ERV_{stamp.strftime('%Y-%M-%d_%H-%M-%S')}.xlsx"
datos = pd.read_csv(ruta_tabla)
#datos = pd.read_excel(ruta_tabla,)
writer = pd.ExcelWriter(outfile, engine='xlsxwriter')

datos = datos.astype({'baja':str, 'mini':str}, errors='raise')
datos.insert(loc=0, column='img', value='')
#datos = datos.dropna(subset='mini')
datos['thumbnail'] = datos['mini'].apply(lambda x: x.split(",")[0] if x.split(",")[0].endswith('.jpg') else False)
#datos = datos[datos['thumbnail'] != False]
datos.to_excel(writer, sheet_name='reporte')
book = writer.book

sheet = writer.sheets['reporte']
datos


In [ ]:
for index, row in datos.iterrows():
    #print(row['ruta_baja'])
    id = row['dc:id']
    print(f"Procesando: {id}")
    path = row['ruta_mini']
    file = row['thumbnail'] 
    if not file:
        print(f"Error: Documento {id} no tiene Thumbnail")
        continue
    src = f"{path}{file}"
    rownum = index + 2
    cell = "A{:1d}".format(rownum)
    print(src)
    ##sheet.insert_image(index,16,path,{'object_position': 1})

    if os.path.isdir(src) == True:
        print("Error con {id}: Es un directorio")
        continue

    try: 
        im = Image.open(src)
    except FileNotFoundError:
        print(f"Error: File not found: {src}")
        continue

    size = im.size
    print(size)
    print(cell)
    sheet.insert_image(cell,src,{'object_position': 2})
    sheet.set_column_pixels(0,0,128)
    sheet.set_row_pixels(index + 1,128)
    print("--- Imagen insertada ----")


In [ ]:
book.close()
writer.save()